## Medical Data Analyzer

In [5]:
# Import Basic Libraries
import pandas as pd
import numpy as np

In [6]:
# Import Spacy for NLP processing
import spacy

In [3]:
# Import Spacy Medical Corpus
import en_core_med7_lg
import en_ner_bc5cdr_md

In [56]:
# Load Dataset
med = pd.read_csv('C:\\datasets\\mtsamples.csv', index_col=0)

#### Make EDA

In [57]:
med.head()

,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, TEMPERATURE: 98.6, BLOO...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [63]:
med.tail()

,description,medical_specialty,sample_name,transcription,keywords
4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",NaN
4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun..."
4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema...",NaN
4997,Mother states he has been wheezing and coughing.,Allergy / Immunology,Asthma in a 5-year-old,"CHIEF COMPLAINT: , This 5-year-old male presen...",NaN
4998,"Acute allergic reaction, etiology uncertain, ...",Allergy / Immunology,Allergy Evaluation Consult,"HISTORY: , A 34-year-old male presents today s...",NaN


In [11]:
# Check Null Values
med.isnull().sum()

description             0
medical_specialty       0
sample_name             0
transcription          33
keywords             1068
dtype: int64

In [12]:
med.shape

(4999, 5)

In [13]:
med.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4999 entries, 0 to 4998
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        4999 non-null   object
 1   medical_specialty  4999 non-null   object
 2   sample_name        4999 non-null   object
 3   transcription      4966 non-null   object
 4   keywords           3931 non-null   object
dtypes: object(5)
memory usage: 234.3+ KB


In [54]:
import re

med['transcription'] = med['transcription'].astype('str')
med['transcription'] = med['transcription'].apply(lambda x: re.sub('(\.,)', ". ", x))

In [13]:
# Load the Medical data
nlp = spacy.load("en_core_med7_lg")

In [14]:
# This function generate anotation for each entities and label
def generate_annotation(texts):
    annotations = []
    for text in texts:
        doc = nlp(text)
        entities = []
        for ent in doc.ents:
            entities.append((ent.start_char, ent.end_char, ent.label_))
        annotations.append((text, {'entities': entities}))
    return annotations

# Extract text entities and labels from the dataset (transcription)
medical_doc = med['transcription'].tolist()

# Let's generate annotations
annotations = generate_annotation(medical_doc)

# Let's print documents and annotations
print("Document:")
print(annotations[0][0]) # first document text
print("Annotations:")
print(annotations[0][1]) # annotation for the first document

Document:
SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up. MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra. ALLERGIES: , She has no known medicine allergies. OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78. HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear. Neck:  Supple without ade

### Analyze Patient Disease

In [64]:
# Load open source Healthcare disease and chemical corpse: en_ner_bc5cdr_md
nlp2 = spacy.load("en_ner_bc5cdr_md")

In [49]:
ner2 = nlp2.get_pipe("ner")
ner2.labels

('CHEMICAL', 'DISEASE')

In [51]:
transcription = med['transcription'].iloc[1]
doc2= nlp2(transcription)

col_dict = {}
s_colours = ['#3cb44b','#e6194B', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
for label, colour in zip(nlp2.pipe_labels['ner'], s_colours):
    col_dict[label] = colour

options = {'ents': nlp2.pipe_labels['ner'], 'colors':col_dict}
spacy.displacy.render(doc2, style = 'ent', jupyter = True, options=options)

def extract_keywords(text):
    doc = nlp(text)
    entities = []
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Lets define our major categories
surgery_keywords = ["surgery", "operation", "procedure", "acute Cholangitis", "surgisis", "appendicitis"]
cardio_pul_keywords = ["heart", "cardiovascular", "pulmonary", "lungs"]
orthopaedic_keywords = ["orthopaedic", "bone", "joint", "fracture"]
neurology_keywords = ["neurology", "nervours system", "brain", "nerve"]
general_med_keywords = ["patient", "complaining", "history", "medical"]
    
transcription = med['transcription'].iloc[1]

entities = extract_keywords(transcription.lower())   
is_surgery = any(keyword in transcription.lower() for keyword in surgery_keywords)
is_cardio_pul = any(keyword in transcription.lower() for keyword in cardio_pul_keywords)
is_orthopaedic = any(keyword in transcription.lower() for keyword in orthopaedic_keywords)
is_neurology = any(keyword in transcription.lower() for keyword in neurology_keywords)
is_general_med = any(keyword in transcription.lower() for keyword in general_med_keywords)
    
print("Summary")
print("Surgery: ", is_surgery)
print("Cardio Pulmonary: ", is_cardio_pul)
print("Orthopaedic: ", is_orthopaedic)
print("Neurology: ", is_neurology)
print("General Medicine: ", is_general_med)

Summary
Surgery:  True
Cardio Pulmonary:  True
Orthopaedic:  True
Neurology:  True
General Medicine:  True


### Analyze the Medications Taken

In [13]:
# Load the Medical data
nlp = spacy.load("en_core_med7_lg")

In [31]:
# We Analyze the 'DOSAGE', 'DRUG', 'DURATION', 'FORM', 'FREQUENCY', 'ROUTE', 'STRENGTH'
ner = nlp.get_pipe("ner")
ner.labels

('DOSAGE', 'DRUG', 'DURATION', 'FORM', 'FREQUENCY', 'ROUTE', 'STRENGTH')

In [16]:
col_dict = {}
s_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
for label, colour in zip(nlp.pipe_labels['ner'], s_colours):
    col_dict[label] = colour

options = {'ents': nlp.pipe_labels['ner'], 'colors':col_dict}

transcription = med['transcription'][0]
doc = nlp(transcription)

spacy.displacy.render(doc, style = 'ent', jupyter = True, options = options)

[(ent.text, ent.label_) for ent in doc.ents]

[('Claritin', 'DRUG'),
 ('Zyrtec', 'DRUG'),
 ('daily', 'FREQUENCY'),
 ('Zyrtec', 'DRUG'),
 ('loratadine', 'DRUG'),
 ('Nasonex', 'DRUG'),
 ('two', 'DOSAGE'),
 ('sprays', 'FORM'),
 ('each', 'DOSAGE'),
 ('for three weeks', 'DURATION')]

In [62]:
from  spacy.matcher import Matcher

# Let's load the model
nlp = spacy.load("en_core_med7_lg")

patterns = [
    [{"ENT_TYPE": "DRUG"}, {"LIKE_NUM": True}, {"IS_ASCII": True}],
    [{"LOWER": {"IN": ["mg", "g", "ml"]}}, {"ENT_TYPE": "DRUG"}],
    [{"ENT_TYPE": "DRUG"}, {"IS_DIGIT": True, "OP": "?"}, {"LOWER": {"IN": ["mg", "g", "ml"]}}]
]

matcher = Matcher(nlp.vocab)
matcher.add("DRUG_DOSE", patterns)

#for transcription in  med['transcription'].iloc[1]:
transcription =  med['transcription'].iloc[0]
doc = nlp(transcription)
matches = matcher(doc)
print("Medications Taken:")
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(span.text)        

Medications Taken:
Nasonex two sprays
